In [1]:
import pyspark
from pyspark.sql.functions import * 
from pyspark.sql.types import *

In [2]:
# In Databricks if we have a structured file(csv) we could use the file to dbfs databricks file systems and convert them to tables. 

#Approach 1:
## We are have upoaded the file to dbfs and converted them to tables using GUI
## The created table is named as mass_train

In [3]:
# using the sqlContext to read the read the table
df = sqlContext.sql("SELECT * from  mass_train")

In [4]:
df.show()

In [5]:
#Approach 2
## Read the file from the dbfs
#location
###/FileStore/tables/FileStore/tables/mass_case_description_train_set.csv
###/FileStore/tables/FileStore/tables/mass_case_description_test_set.csv

sparkDF = sqlContext.read.format("csv").load("/FileStore/tables//FileStore/tables/",header=True,inferSchema=True)

In [6]:
sparkDF.show()

In [7]:
## schema names
sparkDF.printSchema()

In [8]:

csvSchema = StructType([ StructField("patient_id", StringType(), True),
                         StructField("breast_density", IntegerType(), True),
                         StructField("left or right breast", StringType(), True),
                         StructField("image view", StringType(), True),
                         StructField("abnormality id", IntegerType(), True),
                         StructField("abnormality type", StringType(), True),
                         StructField("mass shape", StringType(), True),
                         StructField("mass margins", StringType(), True),
                         StructField("assessment", IntegerType(), True),
                         StructField("pathology", StringType(), True),
                         StructField("subtlety", IntegerType(), True),
                                     ])
# read the csv as stream (the stream transfer is fast as the input csv file has small size) using the read stream function
csv_sdf = spark.readStream.csv("/FileStore/tables//FileStore/tables/",schema=csvSchema)

In [9]:
#check whether the streamed dataset has the same schema type as defined
csv_sdf.schema == csvSchema

In [10]:
#The usual first step in attempting to process the data is to interactively query the data. Let's define a static DataFrame on the files, and give it a table name
#https://docs.databricks.com/spark/latest/structured-streaming/index.html#structured-streaming-python
## When streaming a data, make sure to remove the header as we define them in schema
###/FileStore/tables/data/mass_case_train_set.csv

# Here we are creating a static data frame to check and visualize the dataset
inpData ="/FileStore/tables/data/"

cSchema = StructType([ StructField("patient_id", StringType(), True),
                         StructField("breast_density", IntegerType(), True),
                         StructField("left or right breast", StringType(), True),
                         StructField("image view", StringType(), True),
                         StructField("abnormality id", IntegerType(), True),
                         StructField("abnormality type", StringType(), True),
                         StructField("mass shape", StringType(), True),
                         StructField("mass margins", StringType(), True),
                         StructField("assessment", IntegerType(), True),
                         StructField("pathology", StringType(), True),
                         StructField("subtlety", IntegerType(), True),
                                     ])

staticInputDF = (
  spark
    .read
    .schema(cSchema)
    .csv(inpData)
)

In [11]:
display(staticInputDF)

In [12]:
#only takeing the specific column from a dataset and storing in new view
from pyspark.sql.functions import *      # for window() function

staticCountsDF = (
  staticInputDF
    .groupBy(
       staticInputDF.breast_density, 
       window(staticInputDF.patient_id, "1 hour"))    
    .count()
)
staticCountsDF.cache()

# create temporary views
## Register the DataFrame as table 'static_counts'
staticCountsDF.createOrReplaceTempView("static_counts")

In [13]:
# Streaming of the data

from pyspark.sql.functions import *

# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = (
  spark
    .readStream                       
    .schema(cSchema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .csv(inpData)
)

# Same query as staticInputDF
streamingCountsDF = (                 
  streamingInputDF
    .groupBy(
      streamingInputDF.breast_density, 
      window(streamingInputDF.patient_id, "1 hour"))
    .count()
)

# Is this DF actually a streaming DF?
streamingCountsDF.isStreaming

In [14]:
display(streamingInputDF)

In [15]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table (for testing only in Spark 2.0)
    .queryName("counts")     # counts = name of the in-memory table
    .outputMode("complete")  # complete = all the counts should be in the table
    .start()
)

In [16]:
# spark data frame consists of both train and test data.
display(sparkDF)


In [17]:
sparkDF.count()

In [18]:
#logistics regression

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Logistic').getOrCreate()

from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier as RF
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [19]:
sparkDF.columns

In [20]:
my_col=sparkDF['breast_density',
 'left or right breast',
 'image view',
 'abnormality id',
 'mass shape',
 'mass margins',
 'assessment',
 'subtlety',
 'pathology']

In [21]:
display(my_col)

In [22]:
# check if we have any NULL columns
final_data =  my_col.na.drop()

In [23]:
final_data.count()
#no null rows

In [24]:
left_or_right_indexer = StringIndexer(inputCol='left or right breast',outputCol='leftrightIndex')
left_or_right_encoder = OneHotEncoder(inputCol='leftrightIndex',outputCol='leftrightVector')

In [25]:
image_view_indexer = StringIndexer(inputCol='image view',outputCol='imageViewIndex')
image_View_Vector = OneHotEncoder(inputCol='imageViewIndex',outputCol='imageViewVector')

In [26]:
abnormality_Vector = OneHotEncoder(inputCol='abnormality id',outputCol='abnormalityVector')

In [27]:
mass_shape_indexer = StringIndexer(inputCol='mass shape',outputCol='massShapeIndexer')
mass_shape_vector = OneHotEncoder(inputCol='massShapeIndexer',outputCol='massShapeVector')

In [28]:
mass_margins_Indexer = StringIndexer(inputCol='mass margins',outputCol='massMarginsIndexer')
mass_margins_Vector = OneHotEncoder(inputCol='massMarginsIndexer',outputCol='massMarginVector')

In [29]:
assessment_Vector = OneHotEncoder(inputCol='assessment',outputCol='assessmentVector')
subtlety_Vector = OneHotEncoder(inputCol='subtlety',outputCol='subtletyVector')

In [30]:
assembler = VectorAssembler(inputCols=['leftrightVector','imageViewVector','abnormalityVector','massShapeVector','massMarginVector','assessmentVector','subtletyVector'],
                           outputCol='features')

In [31]:
#Pipeline sets stages for different steps. Here we are dealing with indexing and encoding
from pyspark.ml import Pipeline

In [32]:
pathology_Indexer = StringIndexer(inputCol='pathology',outputCol='pathologyIndexer')

In [33]:
pipeline = Pipeline(stages=[left_or_right_indexer,image_view_indexer,mass_shape_indexer,mass_margins_Indexer,
                           left_or_right_encoder,image_View_Vector,abnormality_Vector,mass_shape_vector,mass_margins_Vector,
                           assessment_Vector,subtlety_Vector,assembler,pathology_Indexer])

In [34]:
alldata = pipeline.fit(final_data).transform(final_data)

In [35]:
data = alldata['features','pathologyIndexer']

In [36]:
train_data, test_data = data.randomSplit([0.7,0.3])

In [37]:
acc_val = MulticlassClassificationEvaluator(labelCol='pathologyIndexer',metricName='accuracy')

In [38]:
#Apply logistic Regression
log_reg = LogisticRegression(featuresCol='features',labelCol='pathologyIndexer')
fit = log_reg.fit(train_data)
transformed = fit.transform(test_data)
print("Accuracy of Logistic Regression:",acc_val.evaluate(transformed))

In [39]:
#Apply Randomn forest
rf = RF(labelCol='pathologyIndexer', featuresCol='features',numTrees=200)
fit = rf.fit(train_data)
transformed = fit.transform(test_data)
print("Accuracy of Randomn Forest:",acc_val.evaluate(transformed))

In [40]:
#Apply Decision Tree
dtc = DecisionTreeClassifier(featuresCol='features',labelCol='pathologyIndexer')
fit = dtc.fit(train_data)
transformed = fit.transform(test_data)
print("Accuracy of Randomn Forest:",acc_val.evaluate(transformed))